In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive



/var/folders/bx/n7x8tlw17p9_m_8d4g08505w0000gn/T/ipykernel_54690/2758595007.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/fluffysnail/Library/CloudStorage/OneDrive-Personal/dtu/current/42186_modelml/42186_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
# df_m1 = pd.read_csv('../df_matches_clean.csv')
df_m1 = pd.read_csv('../data/model1_X_train.csv')
df_m1_test = pd.read_csv('../data/model1_X_test.csv')  


#Make coach id to index mapping
unique_coaches = pd.concat([df_m1['team1_coach_id'], df_m1['team2_coach_id']]).unique()
coaches_sorted = np.sort(unique_coaches)

id_to_index = {id_: idx for idx, id_ in enumerate(coaches_sorted)}
index_to_id = {index: id_ for id_, index in id_to_index.items()}

def map_index_to_coach_id(index):
    return index_to_id.get(index, None)


foo = df_m1['team1_coach_id'].map(id_to_index)
bar = df_m1['team2_coach_id'].map(id_to_index)
baz = df_m1['team1_win']
df_train = pd.concat([foo, bar, baz], axis=1)
df_train.columns = ['coach1_id', 'coach2_id', 'coach1_win']

df_train['coach1_win'] = df_train['coach1_win'] + 1

#Because of model change from (-1,0,1) -> (0,1,2)
print("test")

test


In [5]:
#make function for f:id -> coach_id
    #This should be a mu and a sigma
#in model: make function for skills f2:f(id) -> coach_id
#make results so that they have results and coach_id's ? 

coaches_sorted = np.sort(unique_coaches)
#Initialize skill mu and sigma
coach_mu = torch.zeros(len(coaches_sorted))
coach_sigma = torch.ones(len(coaches_sorted))




In [13]:
def model2(pd_data):
    # results = pd_data['team1_win']
    # coach1 = pd_data['team1_coach_id']
    # coach2 = pd_data['team2_coach_id']
    # c1_mu_idx = coach_mu.searchsorted(coach1)
    # c1_sigma_idx = coach_sigma.searchsorted(coach1)
    # c2_mu_idx = coach_mu.searchsorted(coach2)
    # c2_sigma_idx = coach_sigma.searchsorted(coach2)

    #Make entire skillset for all coaches, in order to save latent variables
    all_skills = dist.MultivariateNormal("coach_skills", loc=coach_mu, covariance_matrix=torch.diag(coach_sigma), validate_args=True)
    c1_skill = all_skills[c1_mu_idx]
    c2_skill = all_skills[c2_mu_idx]
    # all_skills[c1_mu_idx] = pyro.sample("c1_skill", dist.Normal(coach_mu[c1_mu_idx], coach_sigma[c1_sigma_idx]))


    #Pick out the two gaussian distributions for the two coaches

In [7]:
def skill_model(coach_results):
    skills = {}
    
    # Setting up hyperpriors for coach mu
    mu_hyper_mu = pyro.sample("mu_hyper_mu", dist.Normal(0, 10))
    mu_hyper_sigma = pyro.sample("mu_hyper_sigma", dist.HalfNormal(0, 1))
    
    # Setting up hyperpriors for coach sigma
    sigma_hyper_scale = pyro.sample("sigma_hyper_scale", dist.HalfCauchy(1))

    with pyro.plate("coaches_plate", len(unique_coaches)):
        for i, coach_id in enumerate(unique_coaches):            
            # Each coach's mu has a normal prior
            mu = pyro.sample(f"mu_{coach_id}", dist.Normal(mu_hyper_mu, mu_hyper_sigma))
            # Each coach's sigma has a HalfCauchy prior
            sigma = pyro.sample(f"sigma_{coach_id}", dist.HalfCauchy(sigma_hyper_scale))
            skills[coach_id] = pyro.sample(f"skill_{coach_id}", dist.Normal(mu, sigma))
    
    with pyro.plate("games_plate", len(coach_results)):
        team1_coaches = coach_results['team1_coach_id'].values
        team2_coaches = coach_results['team2_coach_id'].values
        results = coach_results['team1_win'].values
        
        for i in range(len(coach_results)):
            skill_diff = skills[team1_coaches[i]] - skills[team2_coaches[i]]
            logits = torch.tensor([skill_diff, 0.0, -skill_diff])
            outcome_probs = torch.softmax(logits, 0)
            pyro.sample(f"result_{i}", dist.Categorical(outcome_probs), obs=torch.tensor(results[i]))



In [8]:
def guide(coach_results):
    for coach_id in unique_coaches:
        mu_loc = pyro.param(f"{coach_id}_mu_loc", torch.tensor(0.0))
        mu_scale = pyro.param(f"{coach_id}_mu_scale", torch.tensor(1.0), constraint=dist.constraints.positive)
        sigma_loc = pyro.param(f"{coach_id}_sigma_loc", torch.tensor(1.0))
        sigma_scale = pyro.param(f"{coach_id}_sigma_scale", torch.tensor(0.05), constraint=dist.constraints.positive)
        
        mu = pyro.sample(f"mu_{coach_id}", dist.Normal(mu_loc, mu_scale))
        sigma = pyro.sample(f"sigma_{coach_id}", dist.LogNormal(sigma_loc, sigma_scale))


In [15]:
optimizer = Adam({"lr": 0.01})
svi = SVI(model2, guide, optimizer, loss=Trace_ELBO())

# Training loop
num_steps = 5000
batch_size = 377 #goes into len(df_train) which is 76154
for step in range(num_steps):
    #Code here
    #Feed in batches (e.g. batches of observations)
    loss = svi.step(df_train)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")

    #stop when it stops improving

# Fetch updated parameter estimates
updated_params = {key: pyro.param(key).item() for key in pyro.get_param_store().keys()}
# print(updated_params)


AttributeError: 'Tensor' object has no attribute 'searchsorted'